##KLUE-BERT를 이용한 개체명 인식
한국어BERT를 이용하여 개체명 인식 문제를 풀어보겠습니다.

In [1]:
!pip install transformers

In [2]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e5f0fd55401e93c9949df38a7c8cdf11ec518312a7ead1c81597f24140a25a9a
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


###1. 데이터 로드

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf
import urllib.request

훈련 데이터와 테스트 데이터 그리고 레이블 정보를 저자의 깃허브에서 다운로드 합니다.

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_train_data.csv", filename="ner_train_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_test_data.csv", filename="ner_test_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/dataset/ner_label.txt", filename="ner_label.txt")

('ner_label.txt', <http.client.HTTPMessage at 0x7b9c78776cd0>)

In [5]:
train_ner_df = pd.read_csv("ner_train_data.csv")
test_ner_df = pd.read_csv("ner_test_data.csv")

In [6]:
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [7]:
print("학습 데이터 샘플 개수: ", len(train_ner_df))
print("테스트 데이터 샘플 개수: ", len(test_ner_df))

학습 데이터 샘플 개수:  81000
테스트 데이터 샘플 개수:  9000


훈련 데이터와 테스트 데이터에서 문장과 레이블을 각각 분리하여 저장합니다.


In [8]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

In [9]:
print(train_data_sentence[2])
print(train_data_label[2])

['슈나이더의', '프레젠테이션은', '말', '청중을', '위한', '특별한', '쇼다', '.']
['PER-B', 'O', 'O', 'CVL-B', 'O', 'O', 'O', 'O']


이 데이터는 형태소 단위가 아니라 어절 단위(띄어쓰기 단위)로 개체명 인식 레이블이 태깅되었다는 특징이 있습니다. ner_label.txt파일을 읽어서 개체명 태깅 정보의 종류를 확인해봅시다.

In [10]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]
print('개 체 명 태 깅 정 보 :', labels)

개 체 명 태 깅 정 보 : ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


개체명 태깅 정보를 저장한 labels 리스트로부터 각 개체명 태깅 정보와 정수를 맵핑하는 딕셔너리를 만
듭니다. tag_to_index 는 개체명 태깅 정보를 key, 정수를 value 로 하는 딕셔너리이며 index_to_tag 는
그 반대 형태를 가지는 딕셔너리입니다.

In [11]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [12]:
print(tag_to_index)
print(index_to_tag)

{'O': 0, 'PER-B': 1, 'PER-I': 2, 'FLD-B': 3, 'FLD-I': 4, 'AFW-B': 5, 'AFW-I': 6, 'ORG-B': 7, 'ORG-I': 8, 'LOC-B': 9, 'LOC-I': 10, 'CVL-B': 11, 'CVL-I': 12, 'DAT-B': 13, 'DAT-I': 14, 'TIM-B': 15, 'TIM-I': 16, 'NUM-B': 17, 'NUM-I': 18, 'EVT-B': 19, 'EVT-I': 20, 'ANM-B': 21, 'ANM-I': 22, 'PLT-B': 23, 'PLT-I': 24, 'MAT-B': 25, 'MAT-I': 26, 'TRM-B': 27, 'TRM-I': 28}
{0: 'O', 1: 'PER-B', 2: 'PER-I', 3: 'FLD-B', 4: 'FLD-I', 5: 'AFW-B', 6: 'AFW-I', 7: 'ORG-B', 8: 'ORG-I', 9: 'LOC-B', 10: 'LOC-I', 11: 'CVL-B', 12: 'CVL-I', 13: 'DAT-B', 14: 'DAT-I', 15: 'TIM-B', 16: 'TIM-I', 17: 'NUM-B', 18: 'NUM-I', 19: 'EVT-B', 20: 'EVT-I', 21: 'ANM-B', 22: 'ANM-I', 23: 'PLT-B', 24: 'PLT-I', 25: 'MAT-B', 26: 'MAT-I', 27: 'TRM-B', 28: 'TRM-I'}


In [13]:
tag_size = len(tag_to_index)
index_size = len(index_to_tag)

print('개체명 태깅 정보의 개수:', tag_size)
print('정수 인덱스의 개수:', index_size)

개체명 태깅 정보의 개수: 29
정수 인덱스의 개수: 29


###2. 전처리 예시

In [14]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

전처리 과정을 이해해봅시다. 임의로 훈련 데이터 중 1번 인덱스의 문장과 레이블을 선택하고 이에 대해서 전처리를 진행해보겠습니다. 해당 문장과, 레이블, 그리고 레이블에 대해서 정수 인코딩을 진행한 결과는 다음과 같습니다.

In [15]:
sent = train_data_sentence[1]
label = train_data_label[1]
print('문장 :', sent)
print('레이블 :',label)
print('레이블의 정수 인코딩 :',[tag_to_index[idx] for idx in label])
print('문장의 길이 :', len(sent))
print('레이블의 길이 :', len(label))

문장 : ['▶', '쿠마리', '한동수가', '말하는', "'가넷", '&', "에르덴'"]
레이블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레이블의 정수 인코딩 : [0, 1, 2, 0, 1, 0, 1]
문장의 길이 : 7
레이블의 길이 : 7


위 문장과 레이블을 예시로 들어 BERT토크나이저를 적용하는 과정을 이해해봅시다. 이미 위와 같이 문장에 단어 토큰화가 진행되어져 있는 상황에서 BERT토크나이저를 적용하기 위해서는 각 단어에 대해서 BERT토크나이저를 사용하여 단어를 서브워드로 분리합니다. 위 문장에 BERT토크나이저를 적용한 결과는 다음과 같습니다.

In [16]:
tokens = []

for one_word in sent:
  # 각 단 어 에 대 해 서 서 브 워 드 로 분 리.
  # ex) one_word = '쿠 마 리' ===> subword_tokens = ['쿠', '##마 리']
  # ex) one_word = '한 동 수 가' ===> subword_tokens = ['한 동', '##수', '##가']
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)

print('BERT 토 크 나 이 저 적 용 후 문 장 :',tokens)
print('레 이 블 :', label)
print('레 이 블 의 정 수 인 코 딩 :',[tag_to_index[idx] for idx in label])
print('문 장 의 길 이 :', len(tokens))
print('레 이 블 의 길 이 :', len(label))

BERT 토 크 나 이 저 적 용 후 문 장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레 이 블 : ['O', 'PER-B', 'PER-I', 'O', 'PER-B', 'O', 'PER-B']
레 이 블 의 정 수 인 코 딩 : [0, 1, 2, 0, 1, 0, 1]
문 장 의 길 이 : 16
레 이 블 의 길 이 : 7


‘쿠마리’ 가’ 쿠’ 와’## 마리’ 로 분리되는 등 단어들이 서브워드로 분리되었습니다. 이제 문장의 길이가 길어지면서 레이블의 길이와 달라지게 됩니다. 레이블의 길이도 문장의 길이와 일치하도록 추가적인 처리를 진행해야 합니다. ‘쿠마리’ 의 레이블은’PER‐B’ 였습니다. 그렇다면’ 쿠’ 와’## 마리’ 의 레이블은 어떻게 해야할까요?
이 경우 첫번째 서브워드에 대해서만 기존의 레이블을 부여하고 뒤에 생겨난 서브워드들에 대해서는 레이블을 주지 않는 방법이 있습니다. 가령, 단어 ‘쿠마리’ 가’PER‐B’ 의 레이블을 가지고 있었다면, 분리된 [’쿠’, ’## 마리’] 에 대해서’ 쿠’ 에는’PER‐B’ 를 부여하고 그 뒤의 서브워드인’## 마리’ 에 대해서는 레이블을
주지 않는 것입니다. 그 방법으로는 레이블을 정수 인코딩하는 과정에서 첫번째 서브워드가 아닌 경우에
는 ‐100 을 부여하는 방식을 사용합니다.

In [17]:
tokens = []
labels_ids = []

for one_word, label_token in zip(train_data_sentence[1], train_data_label[1]):
  subword_tokens = tokenizer.tokenize(one_word)
  tokens.extend(subword_tokens)
  labels_ids.extend([tag_to_index[label_token]]+ [-100] * (len(subword_tokens) -1))

print('토 큰 화 후 문 장 :',tokens)
print('레 이 블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx in labels_ids])
print('레 이 블 의 정 수 인 코 딩 :', labels_ids)
print('문 장 의 길 이 :', len(tokens))
print('레 이 블 의 길 이 :', len(labels_ids))

토 큰 화 후 문 장 : ['▶', '쿠', '##마리', '한동', '##수', '##가', '말', '##하', '##는', "'", '가', '##넷', '&', '에르', '##덴', "'"]
레 이 블 : ['O', 'PER-B', '[PAD]', 'PER-I', '[PAD]', '[PAD]', 'O', '[PAD]', '[PAD]', 'PER-B', '[PAD]', '[PAD]', 'O', 'PER-B', '[PAD]', '[PAD]']
레 이 블 의 정 수 인 코 딩 : [0, 1, -100, 2, -100, -100, 0, -100, -100, 1, -100, -100, 0, 1, -100, -100]
문 장 의 길 이 : 16
레 이 블 의 길 이 : 16


‘레이블의 정수 인코딩’ 결과를 보면 ‘쿠’ 에 대해서는 PER‐B 에 해당하는 정수 1 을 부여하였지만, # 마리
에 대해서는 ‐100 을 부여하였습니다. 마찬가지로 기존에는’ 한동수가’ 에 PER‐I 가 부여되어 있었으나, 서
브워드 토큰화 과정에서 [’ 한동’, ’## 수’, ’## 가’] 로 분리되었고, 첫번째 서브워드인’ 한동’ 에는 PER‐I 에
해당하는 정수인 2 를 부여하지만 그 뒤의 서브워드들인’## 수’, ’## 가’ 에 대해서는 ‐100 을 부여합니다. ’
말하는’ 은 레이블이’O’ 였으나 [‘말’, ‘##’ 하,’## 는’] 으로 분리되었고’ 말’ 에 대해서는’O’ 에 해당하는 정수
인 0 을 부여하지만, 그 뒤의 서브워드 들에 대해서는 ‐100 을 부여합니다.
‐100 을 부여하고나서는 실질적으로 학습할 때는 해당 레이블에 대해서는 학습을 무시하는 정책을 취할
예정입니다. 정확히는 ‐100 을 레이블에서는 패딩 토큰 [PAD] 로 사용합니다. 레이블에 대해서는 문장의
길이를 맞추는 패딩을 진행할 때도 ‐100 을 사용하겠습니다. 그리고 이후 손실 함수에서 ‐100 을 무시하도
록 추가 처리를 진행하겠습니다.
레이블에 대한 전처리를 이해하였다면, 이제 문장과 레이블에 대한 정수 인코딩. 그리고 세그먼트 인코딩
과 어텐션 마스크까지 생성하는 함수 convert_examples_to_features 를 구현합니다.

In [18]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):

  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id

  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  for example, label in tqdm(zip(examples, labels), total=len(examples)):
    tokens = []
    labels_ids = []
    for one_word, label_token in zip(example, label):
      # 하나의 단어에 대해서 서브워드로 토큰화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서브워드 중 첫번째 서브워드만 개체명 레이블을 부여 하고 그 외에는 -100으로 채운다.

      labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

    # [CLS]와 [SEP]를 후에 추가 할 것을 고려 하여 최대 길이를 초과 하는 샘플 의 경우 max_seq_len - 2의 길이로 변환.
    # ex) max_seq_len = 64라면 길이가 62 보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환.
    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

    # [SEP]를 추가 하는 코드
    # 1. 토큰화 결과의 맨 뒷 부분에 [SEP]토큰 추가
    # 2. 레이블에도 맨 뒷 부분에 -100 추가.
    tokens += [sep_token]
    labels_ids += [pad_token_id_for_label]

    # [CLS]를 추가 하는 코드
    # 1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
    # 2. 레이블의 맨 앞 부분에도 -100 추가.
    tokens = [cls_token] + tokens
    labels_ids = [pad_token_id_for_label] + labels_ids

    # 정수 인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)

    # 어텐션 마스크 생성
    attention_mask = [1] * len(input_id)

    # 정수 인코딩에 추가 할 패딩 길이 연산
    padding_count = max_seq_len - len(input_id)

    # 정수 인코딩, 어텐션 마스크에 패딩 추가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)

    # 세 그 먼 트 인 코 딩.
    token_type_id = [pad_token_id_for_segment] * max_seq_len

    # 레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
    label = labels_ids + ([pad_token_id_for_label] * padding_count)
    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token type length {}vs {}".format(len(token_type_id), max_seq_len)
    assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.asarray(data_labels, dtype=np.int32)
  return (input_ids, attention_masks, token_type_ids), data_labels

훈련 데이터와 테스트 데이터에 대해서 전처리를 진행합니다. 단, 문장의 최대 길이는 임의로 128로 지정하였습니다.


In [19]:
X_train, y_train = convert_examples_to_features(train_data_sentence,
                                                train_data_label, max_seq_len=128, tokenizer=tokenizer)
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label,
                                              max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:03<00:00, 2535.87it/s]


훈련 데이터의 첫번째 샘플을 예시로 보겠습니다. 기존에 주어졌던 원문과 레이블이 BERT토크나이저 적용후 어떻게 변경되었는지 보고, 그후에 실질적으로 BERT의 입력이 되는 정수 인코딩 결과는 어떻게 되었는지 살펴봅시다.

In [20]:
print('기 존 원 문 :', train_data_sentence[0])
print('기 존 레 이 블 :', train_data_label[0])
print('-' * 50)
print('토 큰 화 후 원 문 :', [tokenizer.decode([word]) for word in X_train[0][0]])
print('토 큰 화 후 레 이 블 :', ['[PAD]' if idx == -100 else index_to_tag[idx] for idx
in y_train[0]])
print('-' * 50)
print('정 수 인 코 딩 결 과 :', X_train[0][0])
print('정 수 인 코 딩 레 이 블 :', y_train[0])

기 존 원 문 : ['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
기 존 레 이 블 : ['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
--------------------------------------------------
토 큰 화 후 원 문 : ['[CLS]', '정은', '씨', '##를', '힘들', '##게', '한', '가스', '##나', '##그', ',', '가만', '##둘', '수', '없', '##겠', '##죠', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

‘토큰화 후 레이
블’ 과’ 정수 인코딩 레이블’ 결과를 비교해서 보면, 첫번째 서브워드에는 기존의 레이블을 부여하고, 그 뒤
의 서브워드들에 대해서는 ‐100 을 부여하였습니다. 가령, ’ 씨를’ 이 [‘씨’, ’##’ 를] 로 분리되면서 각각 0 과
‐100 의 레이블이 부여되었습니다. 그 외에는 [CLS] 토큰과 [SEP] 토큰의 위치에 대해서도 개체명 예측이
의미없으므로 ‐100 을 부여했습니다. 세그먼트 인코딩과 어텐션 마스크에 대해서 출력해봅시다.

In [21]:
print('정수 인코딩: ', X_train[0][0])
print('세그먼트 인코딩: ', X_train[2][0])
print('어텐션 마스크: ', X_train[1][0])

정수 인코딩:  [    2 17915  1370  2138  4390  2318  1891  5809  2075  2029    16  6836
  3056  1295  1415  2918  2321    18     3     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
세그먼트 인코딩:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

세그먼트 인코딩과 어텐션 마스크의 경우 네이버 영화 리뷰 분류와 같은 이전 문제들과 부여 방식이 동일 합니다. 세그먼트 인코딩에서는 개체명 인식은 2개 이산의 문장을 구분할 필요가 없는 문제이므로 전부 0으로 채워줍니다. 어텐션 마스크의 경우에는 [CLS]부터 [SEP]까지 1을 부여하고, [SEP]이후의 [PAD]에 대해서 0을 부여 합니다.

### 4. 모델링
개체명 인식을 위한 모델을 설계해봅시다. 텍스트 분류를 수행할 때는 BERT 의 출력인 outputs 에서
인덱스 1 로 접근하였습니다. outputs[1] 은 [CLS] 토큰에 접근하는 방법으로 Many‐to‐One 문제에 해
당하는 텍스트 분류를 풀 때 사용합니다. 반면, 이번에는 모든 입력에 대해서 출력을 수행해야 하는
Many‐to‐Many 문제에 해당하므로 outputs[0] 을 사용하여 문제를 풀어야 합니다.
outputs[0] 과 연결되는 출력층에는 레이블의 개수인 num_labels 를 전달합니다. 이번에는 다중 클래스
분류 문제임에도 출력층에 소프트맥스 함수를 사용하지 않았는데, 이번에는 출력층에 소프트맥스 함수
를 누락시킨 후에 손실 함수에서 이를 처리하도록 하는 구현 방식을 사용해보기 위함입니다.


In [22]:
class TFBertForTokenClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForTokenClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,
                                            kernel_initializer=tf.keras.
                                            initializers.TruncatedNormal
                                             (0.02),
                                            name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
    token_type_ids=token_type_ids)

    # 전 체 시 퀀 스 에 대 해 서 분 류 해 야 하 므 로 outputs[0]임 에 주 의
    all_output = outputs[0]
    prediction = self.classifier(all_output)
    return prediction

###5. 손실 함수에서 -100레이블은 제외시키기(예시 코드)

앞서 레이블이 ‐100 인 경우에 대해서는 손실 함수에서 오차를 구현하지 않도록 할 예정이라고 언급했습
니다. 다시 말해서 레이블이 ‐100 인 경우에 대해서는 학습에 반영하지 않고자 합니다. 이를 어떻게 구현
할 수 있는지 임의의 다중 클래스 분류 모델을 가정하고 예시 코드를 통해서 살펴봅시다.
labels 를 실제값에 해당하는 레이블. logits 을 어떤 모델의 예측값이라고 가정해봅시다. 우리가 지금까
지 다뤄왔던 다중 클래스 분류 모델은 처음부터 정수 레이블을 예측하지 않습니다. 기본적으로 각 레이블
이 정답일 확률값을 예측합니다. 출력층에서 소프트맥스 함수를 통과하고 나면 이 값들의 총 합은 1 이 되
기 때문입니다. 가정하기를 현재 가정하고 있는 모델이 예측해야 하는 레이블의 개수가 3 개라고 해봅시
다. 다시 말해 logits 의 각 예측값은 3 차원 벡터입니다.

In [23]:
# tf.constant()는 TensorFlow에서 변하지 않는 상수 텐서를 생성할 때 사용하는 함수
labels = tf.constant([[-100, 2, 1, -100]])
logits = tf.constant([[[0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15], [0.4, 0.5, 0.1]]])

In [24]:
active_loss = tf.reshape(labels, (-1,)) != -100
print(active_loss)

tf.Tensor([False  True  True False], shape=(4,), dtype=bool)


active_loss 는 레이블에서 ‐100 인 경우에는 False ‐100 이 아닌 경우에는 True 의 값을 가집니다. 이제
active_loss 를 활용하여 logits 에서 ‐100 의 위치를 가지는 경우의 예측값은 제거합니다. 그 결과는 다음
과 같습니다.

In [25]:
reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
print(reduced_logits)

tf.Tensor(
[[0.06 0.04 0.9 ]
 [0.75 0.1  0.15]], shape=(2, 3), dtype=float32)


reduced_logits 을 보면 기존의 4 개의 예측값인 [0.8, 0.1, 0.1], [0.06, 0.04, 0.9], [0.75, 0.1, 0.15], [0.4, 0.5,
0.1] 중에서 두번째 위치의 값인 [0.06, 0.04, 0.9] 과 세번째 위치의 값인 [0.75, 0.1, 0.15] 값만이 살아남은
것을 볼 수 있는데, 이는 active_loss 에서 두번째 위치와 세번째 위치의 값만이 True 였기 때문입니다. 다
시 말해 레이블이 ‐100 이 아닌 위치의 예측값만 남긴 것입니다.
마찬가지로 labels 또한 ‐100 이 아닌 위치의 실제값만 남겨봅시다.

In [26]:
labels = tf.boolean_mask(tf.reshape(labels, (-1)), active_loss)
print(labels)

tf.Tensor([2 1], shape=(2,), dtype=int32)


동일하게 두번째 위치의 값과 세번째 위치의 값만 살아남고 ‐100 의 값을 가졌던 첫번째 값과 네번째 값은
제거되었습니다. 다시 정리하겠습니다. 다음과 같이 레이블. 즉, 실제값과 예측값이 있다고 가정해봅시다.

• 실제값: [‐100, 2, 1, ‐100]

• 예측값: [예측값 1, 예측값 2, 예측값 3, 예측값 4]

레이블의 값이 ‐100 인 경우에는 오차를 구하고 싶지 않습니다. 레이블의 값이 ‐100 이 인 경우에는 실제
값과 예측값에 대해서 모두 값을 제거해버립니다.

• 실제값: [2, 1]

• 예측값: [예측값 2, 예측값 3]

이제 레이블의 값이 ‐100 인 경우는 제거되었으니 실제값과 예측값에 대해서 오차를 구하면 레이블의 값
이 ‐100 인 경우에 대해서는 오차가 계산되지 않으며 학습에도 반영되지 않게 됩니다.

###6. 손실 함수 구현하기
위의 예시 코드를 통해 이해한 레이블의 값이 -100인 경우를 학습시에 무시하는 방법을 적용하여 손실함수를 구현합니다.

In [27]:
def compute_loss(labels, logits):

  # 다 중 클 래 스 분 류 문 제 에 서 소 프 트 맥 스 함 수 미 사 용 시 from_logits=True로 설 정.
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

  # -100의 값 을 가 진 정 수 에 대 해 서 는 오 차 를 반 영 하 지 않 도 록 labels를 수 정.
  active_loss = tf.reshape(labels, (-1,)) != -100

  # activa_loss로 부 터 reduced_logits과 labels를 각 각 얻 는 다.
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
  return loss_fn(labels, reduced_logits)

손실 함수로는 다중 클래스 분류 문제이므로 우리가 지금까지 다중 클래스 분류 문제에서 사용해왔던
것처럼 SparseCategoricalCrossentropy 를 사용합니다. 다만, 우리는 ‐100 이라는 숫자를 레이블에 대한
[PAD] 토큰으로 간주하여 loss 를 구하지 않는 것으로 가정하고 전처리를 진행하였으므로 손실 함수에서
도 이를 반영해주어야 합니다.
한 가지 더 고려해야할 점은 앞서 만든 TFBertForTokenClassification 모델에서 출력층에 소프트맥스 함
수를 사용하지 않아 예측 벡터의 총 합이 1 이 되지 않은 상태라는 점입니다. 이 경우 SparseCategorical‐
Crossentropy 에서 from_logits 를 True 로 지정하면 예측값이 소프트맥스 함수를 통과하지 않았음을 고
려하고 오차를 구합니다.
이에 대한 설명은 아래의 SparseCategoricalCrossentropy 의 공식 문서에서 확인할 수 있습니다.
https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
공식 문서에 따르면 from_logits 의 값을 지정해주지 않으면 기본값은 False 로 SparseCategorical‐Crossentropy 는 기본적으로 예측값이 소프트맥스 함수값을 통과한 상태임을 가정합니다. 하지만 예측
값에 소프트맥스 함수를 통과시키지 않은 경우라면, from_logits=True 를 해주면 이를 감안하여 오차를
구하게 됩니다.

In [28]:
model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=compute_loss)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClas

한 에포크가 끝날 때마다 테스트 데이터에 대해서 F1 score 를 계산하여 학습이 과적합되고 있지는 않
은지 판단하고자 합니다. 이를 위해서 tf.keras.callbacks.Callback 를 상속받아서 커스텀 콜백 클래스인
F1score 를 구현했습니다.
학습 중 모델의 동작을 정의하는 커스텀 콜백을 구현하는 방식에 대해서는 아래의 링크를 참고하여 구현
했습니다.
커스텀 콜백 구현 방법: https://www.tensorflow.org/guide/keras/custom_callback?hl=ko
커스텀 콜백 클래스에 on_epoch_end 라는 함수를 정의하면, 에포크가 끝날 때마다 해당 함수가 호출됩
니다.

In [29]:
class F1score(tf.keras.callbacks.Callback):
  def __init__(self, X_test, y_test):
    self.X_test = X_test
    self.y_test = y_test

  def sequences_to_tags(self, label_ids, pred_ids):
    label_list = []
    pred_list = []

    for i in range(0, len(label_ids)):
      label_tag = []
      pred_tag = []

      # 레 이 블 의 값 이 -100인 경 우 는 F1 score 계 산 시 에 도 제 외
      # ex) 레 이 블 디 코 딩 과 정
      # label_index : [1 -100 2 -100] ===> [1 2] ===> label_tag : [PER-B PER-I]
      for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
        if label_index != -100:
          label_tag.append(index_to_tag[label_index])
          pred_tag.append(index_to_tag[pred_index])

      label_list.append(label_tag)
      pred_list.append(pred_tag)

    return label_list, pred_list

  # 에 포 크 가 끝 날 때 마 다 실 행 되 는 함 수}
  def on_epoch_end(self, epoch, logs={}):
    y_predicted = self.model.predict(self.X_test)
    y_predicted = np.argmax(y_predicted, axis = 2)

    label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

    score = f1_score(label_list, pred_list, suffix=True)
    print(' - f1: {:04.2f}'.format(score * 100))
    print(classification_report(label_list, pred_list, suffix=True))

f1 스코어를 계산하기 위해서는 예측값과 레이블이 정수 시퀀스가 아니라 개체명 태깅 정보들의 시퀀스
이어야만 합니다. 정수 시퀀스로부터 개체명 태깅 정보 시퀀스로 변환해주는 함수가 sequences_to_tags
라는 함수입니다.
해당 함수의 동작 방식을 살펴봅시다. 예를 들어 실제값에 해당하는 레이블 (y_test) 이 [‐100 1 ‐100 2 ‐100
‐100] 이고 예측값 (y_predicted) 이 [0 1 0 2 0 0] 이라고 해봅시다. 우리는 여기서 레이블의 값이 ‐100 인
경우에 대해서는 고려하지 않으며 레이블의 값이 ‐100 인 위치에 대해서 레이블과 예측값 모두 제외시킵
니다. 이에 따라 레이블과 예측값은 각각 [1 2], [1 2] 가 됩니다. 이를 개체명 태깅 정보 시퀀스로 변환하면
실제값과 예측값은 [PER‐B, PER‐I] 와 [PER‐B, PER‐I] 가 됩니다. 그리고 이 결과를 f1 score 계산에 반영합
니다. 이 경우 예측값이 실제값을 모두 정확하게 맞춘 경우가 되겠습니다.
배치 크기는 32, 3 에포크 학습합니다.

In [30]:
f1_score_report = F1score(X_test, y_test)
model.fit(X_train, y_train, epochs = 3, batch_size=32, callbacks=[f1_score_report])

Epoch 1/3
282/282 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step
 - f1: 58.86


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         AFW       0.38      0.07      0.12       394
         ANM       0.47      0.07      0.12       701
         CVL       0.57      0.46      0.51      5758
         DAT       0.69      0.60      0.64      2521
         EVT       0.39      0.31      0.34      1094
         FLD       0.00      0.00      0.00       228
         LOC       0.63      0.49      0.55      2126
         MAT       0.00      0.00      0.00        12
         NUM       0.77      0.72      0.74      5590
         ORG       0.59      0.58      0.58      4086
         PER       0.71      0.72      0.72      4426
         PLT       0.00      0.00      0.00        34
         TIM       0.42      0.11      0.18       314
         TRM       0.58      0.29      0.38      1964

   micro avg       0.65      0.54      0.59     29248
   macro avg       0.44      0.31      0.35     29248
weighted avg       0.63      0.54      0.57     29248

2532/2532 ━━━━━━━━━━━━━━━

###3. 예측
이제 학습하지 않은 임의의 문장에 대해서 개체명 태깅 정보를 예측해봅시다. 임의의 문장에 대한 예측
을 위해 전처리를 수행하는 함수인 convert_examples_to_features_for_prediction 를 구현합니다. 기본
적으로 해당 함수가 input_ids, attention_masks, token_type_ids 를 준비하는 방식은 학습 단계에서 전
처리를 위해 사용했던 함수인 convert_examples_to_features 와 같습니다. 학습 단계나 테스트 단계나
BERT 의 입력 형식은 동일하기 때문입니다.
예측을 위한 함수인 convert_examples_to_features_for_prediction 를 구현 시 고려해야할 점은 학습 단
계에서는 레이블이 있었으나 임의의 문장에 대한 예측 시에는 레이블이 존재하지 않는다는 점입니다. 앞
서 f1 score 를 계산할 때는 ‐100 의 위치를 레이블인 y_test 로 파악했습니다. 하지만 이번에는 레이블이
존재하지 않으므로 ‐100 의 위치를 기록해두는 label_mask 를 만들어줍니다.

In [33]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer, pad_token_id_for_segment=0,
                                                pad_token_id_for_label=-100):
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id

  input_ids, attention_masks, token_type_ids, label_masks = [], [],[], []

  for example in tqdm(examples):
    tokens = []
    label_mask = []
    for one_word in example:
      # 하나의 단어에 대해서 서브워드로 토큰화
      subword_tokens = tokenizer.tokenize(one_word)
      tokens.extend(subword_tokens)
      # 서브 워드 중 첫번째 서브워드를 제외하고 그 뒤의 서브워드들은  -100으로 채운다.
      label_mask.extend([0] + [pad_token_id_for_label] * (len(subword_tokens) -1))

    # [CLS]와 [SEP]를 후에 추가 할 것을 고려하여 최대 길이를 초과하는 샘플의 경우 max_seq_len - 2의 길이로 변환.
    # ex) max_seq_len = 64라면 길이가 62보다 긴 샘플은 뒷 부분을 자르고 길이 62로 변환.
    special_tokens_count = 2
    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      label_mask = label_mask[:(max_seq_len - special_tokens_count)]

    # [SEP]을 추가하는 코드
    # 1. 토큰화 결과의 맨 뒷 부분에 [SEP]토큰 추가
    # 2. 레이블에도 맨 뒷 부분에 -100추가.
    tokens += [sep_token]
    label_mask += [pad_token_id_for_label]

    # [CLS]를 추가 하는 코드
    # 1. 토큰화 결과의 앞 부분에 [CLS] 토큰 추가
    # 2. 레이블의 맨 앞 부분에도 -100 추가.
    tokens = [cls_token] + tokens
    label_mask = [pad_token_id_for_label] + label_mask

    # 정수 인코딩
    input_id = tokenizer.convert_tokens_to_ids(tokens)

    # 어텐션 마스크 생성
    attention_mask = [1] * len(input_id)

    # 정수 인코딩에 추가할 패딩 길이 연산
    padding_count = max_seq_len - len(input_id)

    # 정수 인코딩, 어텐션 마스크에 패딩 추가
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)

    # 세그먼트 인코딩.
    token_type_id = [pad_token_id_for_segment] * max_seq_len

    #  레이블 패딩. (단, 이 경우는 패딩 토큰의 ID가 -100)
    label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}". format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention_mask length {} vs {}". format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, "Error with token_type_id length {} vs {}". format(len(token_type_id), max_seq_len)
    assert len(label_mask) == max_seq_len, "Error with label_mask length {} vs {}". format(len(label_mask), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    label_masks.append(label_mask)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  label_masks = np.asarray(label_masks, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), label_masks

임의로 테스트 데이터 중 샘플 5 개를 입력으로 사용하여 정상적으로 개체명 인식 모델이 동작하는지 테
스트 해봅시다.

In [34]:
X_pred, label_masks = convert_examples_to_features_for_prediction(
    test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 1999.76it/s]


테스트 데이터의 첫번째 샘플인 [‘라티은‐원윤정,’, ‘휘닉스파크클래식’, ‘프로골퍼’] 에 대해서 con‐
vert_examples_to_features_for_prediction 가 변환한 결과를 확인해봅시다.

In [35]:
print('기 존 원 문 :', test_data_sentence[0])
print('-' * 50)
print('토 큰 화 후 원 문 :', [tokenizer.decode([word]) for word in X_pred[0][0]])
print('레 이 블 마 스 크 :', ['[PAD]' if idx == -100 else '[FIRST]' for idx in label_masks[0]])

기 존 원 문 : ['라티은-원윤정,', '휘닉스파크클래식', '프로골퍼']
--------------------------------------------------
토 큰 화 후 원 문 : ['[CLS]', '라', '##티', '##은', '-', '원', '##윤', '##정', ',', '휘', '##닉스', '##파크', '##클', '##래', '##식', '프로', '##골', '##퍼', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'

레이블 마스크는 BERT 토크나이저가 하나의 단어에 대해서 서브워드로 분리할 경우, 첫번째 서브워드를 제외한 나머지 서브워드들에 대해서는 [PAD] 토큰.  
즉, ‐100 을 부여합니다. 이제 모델이 임의의 문장에
대해서 예측했을 때, 레이블 마스크의 값을 참고하여 첫번째 서브워드가 아닌 뒤의 서브워드들에 대한 예
측값은 무시합니다.

In [37]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer = tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []

    # ex) 모델의 예측값 디코딩 과정
    # 예측값(y_predicted)에서 레이블 마스크 (label_masks)의 값이 -100인 동일 위치의 값을 삭제
    # label_masks : [-100 0 -100 0 -100]
    # y_predicted : [0 1 0 2 0] --> [1 2] --> 최종 예측(pred_tag) : [PER-B PER-I]

    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

훈련 데이터에 존재하지 않았던 임의의 두 개의 문장에 대해서 개체명을 예측해봅시다.

In [38]:
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀 이다'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'
test_samples = [sent1, sent2]
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)
result_list

100%|██████████| 2/2 [00:00<00:00, 2208.11it/s]


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀', 'O'),
  ('이다', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아', 'O'),
  ('있는데', 'O'),
  ('타인은', 'O'),
  ('살고', 'O'),
  ('있어요', 'O')]]